In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Load the datasets
creditcard_data = pd.read_csv('../data/creditcard.csv')
fraud_data = pd.read_csv('../data/Fraud_Data.csv')
ip_data = pd.read_csv('../data/IpAddress_to_Country.csv')

# Display the first few rows of each dataset
print("Credit Card Data Sample:")
print(creditcard_data.head(), "\n")
print("Fraud Data Sample:")
print(fraud_data.head(), "\n")
print("IP Address Data Sample:")
print(ip_data.head(), "\n")

# Checking for missing values
print("Credit Card Data Missing Values:\n", creditcard_data.isnull().sum(), "\n")
print("Fraud Data Missing Values:\n", fraud_data.isnull().sum(), "\n")
print("IP Address Data Missing Values:\n", ip_data.isnull().sum(), "\n")

# Drop rows with missing values
creditcard_data.dropna(inplace=True)
fraud_data.dropna(inplace=True)
ip_data.dropna(inplace=True)

# Removing duplicates
creditcard_data.drop_duplicates(inplace=True)
fraud_data.drop_duplicates(inplace=True)
ip_data.drop_duplicates(inplace=True)

# Convert relevant columns to datetime
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

# Univariate analysis for fraud_data: Distribution of Purchase Value
print("Plotting Distribution of Purchase Value...")
fraud_data['purchase_value'].hist(bins=50)
plt.title('Distribution of Purchase Value')
plt.show()

# Bivariate analysis between purchase_value and fraud class
print("Plotting Purchase Value vs Fraud Class...")
sns.boxplot(x='class', y='purchase_value', data=fraud_data)
plt.title('Purchase Value vs Fraud Class')
plt.show()

# Convert IP addresses to string, handle NaNs, and then to integer format
fraud_data['ip_address'] = fraud_data['ip_address'].astype(str).apply(lambda ip: int(ip.replace('.', '')) if ip != 'nan' else 0)

# Merge with IP Address dataset
merged_data = pd.merge(fraud_data, ip_data, how='left',
                       left_on='ip_address', right_on='lower_bound_ip_address')

# Drop unnecessary columns from merge
merged_data.drop(['lower_bound_ip_address', 'upper_bound_ip_address'], axis=1, inplace=True)

# Display the merged data
print("Merged Data Sample:")
print(merged_data.head(), "\n")

# Feature Engineering: Transaction Frequency & Velocity
fraud_data['transaction_frequency'] = fraud_data.groupby('user_id')['user_id'].transform('count')
fraud_data['transaction_velocity'] = (fraud_data['purchase_time'] - fraud_data['signup_time']).dt.total_seconds() / 3600

# Feature Engineering: Time-based Features
fraud_data['hour_of_day'] = fraud_data['purchase_time'].dt.hour
fraud_data['day_of_week'] = fraud_data['purchase_time'].dt.dayofweek

# Normalize purchase_value and transaction_velocity
scaler = MinMaxScaler()
fraud_data[['purchase_value', 'transaction_velocity']] = scaler.fit_transform(fraud_data[['purchase_value', 'transaction_velocity']])

# One-hot encoding for categorical columns
fraud_data = pd.get_dummies(fraud_data, columns=['source', 'browser', 'sex'], drop_first=True)

# Display the final fraud_data with new features
print("Fraud Data with New Features:")
print(fraud_data.head(), "\n")
